# Machine Learning Approach to Renewable Energy Dynamics

**Auteurs :** Grzegorz Mozdzynski, Hajar Sriri, Jules Chopard  
**Date :** Avril 2025

This notebook analyzes the determining factors of the share of renewable energies in the final energy consumption of countries. It includes data cleaning, functionality selection, model training (Linear, Lasso, Ridge, Random Forest) and robustness testing (Bootstrap).

In [ ]:
# === 1. IMPORT LIBRARIES ===
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import KNNImputer
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
import statsmodels.api as sm


sns.set_theme(style="whitegrid")
%matplotlib inline

## 2. Data Loading and Cleaning
We load the dataset, clean column names, drop unnecessary geographical variables, and handle missing values in the target variable.

In [ ]:
# Load the dataset
# Ensure the file is in the 'data' folder located one level up
try:
    df = pd.read_csv('../data/global-data-on-sustainable-energy-1.csv')
except FileNotFoundError:
    # Fallback if the file is in the same folder
    df = pd.read_csv('global-data-on-sustainable-energy-1.csv')

# Clean column names
df.columns = df.columns.str.strip().str.replace('\n', ' ').str.replace('  ', ' ')

# Drop geographical and irrelevant columns
columns_to_drop = ['Entity','Year', 'Latitude', 'Longitude', 'Land Area(Km2)', 'Density\\n(P/Km2)']
df_clean = df.drop(columns=columns_to_drop)

# Convert to numeric
df_clean = df_clean.apply(pd.to_numeric, errors='coerce')

# Drop rows where the target is missing
target_col = "Renewable energy share in the total final energy consumption (%)"
df_clean = df_clean.dropna(subset=[target_col])

print(f"Dimensions after initial cleaning: {df_clean.shape}")
df_clean.head()

## 3. Exploratory Data Analysis (Correlation)
We analyze the correlation matrix to identify multicollinearity and see which variables are related to our target.

In [ ]:
# Correlation matrix
plt.figure(figsize=(14, 12))
corr_matrix = df_clean.corr()

sns.heatmap(corr_matrix, cmap='coolwarm', center=0, annot=False, linewidths=0.5)
plt.title("Correlation Matrix")
plt.show()

### Dropping Highly Correlated Variables
Based on the matrix, we drop certain variables (like electricity from fossil fuels) to avoid redundancy.

In [ ]:
# Drop correlated features
cols_correlated = ['Access to clean fuels for cooking','Electricity from fossil fuels (TWh)']
df_clean = df_clean.drop(columns=cols_correlated)

# Show correlations with the target
target_corr = corr_matrix[target_col].sort_values(ascending=False)
print("\nCorrelation of features with the target:")
print(target_corr)

## 4. Missing Value Imputation (KNN)
We use K-Nearest Neighbors imputation (K=3) to fill in missing data while preserving local structures.

In [ ]:
# Define X and y
X = df_clean.drop(columns=[target_col])
y = df_clean[target_col]

# Temporary standardization for KNN
scaler = StandardScaler()
X_scaled_temp = scaler.fit_transform(X)

# Apply KNNImputer (k=3)
imputer = KNNImputer(n_neighbors=3)
X_imputed = imputer.fit_transform(X_scaled_temp)

# Create imputed DataFrame
X = pd.DataFrame(X_imputed, columns=X.columns)

print("Imputation complete.")

## 5. Feature Selection (Mixed Stepwise)
We use a "stepwise" selection (mixing forward and backward) to find the best subset of variables maximizing the R² score.

In [ ]:
X_temp = pd.DataFrame(X)
lr_fs = LinearRegression()

# Apply mixed selection (SFS)
sfs = SFS(lr_fs,
          k_features='best',
          forward=True,
          floating=True,
          scoring='r2',
          cv=5)

sfs = sfs.fit(X_temp, y)

# List of selected features
selected_features = list(sfs.k_feature_names_)
print("Selected features:", selected_features)

# Reduce X to selected features
X = X_temp[selected_features]

## 6. Model Training and Evaluation
We compare here:
1.  **Linear Regression**
2.  **Lasso (L1)**
3.  **Random Forest**

We also check the statistical significance of coefficients via OLS (Statsmodels).

In [ ]:
# Train/Test Split (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize models
lr = LinearRegression()
lasso = LassoCV(cv=5, random_state=42)
rf = RandomForestRegressor(random_state=42)

# Training
lr.fit(X_train, y_train)
lasso.fit(X_train, y_train)
rf.fit(X_train, y_train)

# OLS Statistical Summary
X_train_const = sm.add_constant(X_train.reset_index(drop=True))
y_train_reset = y_train.reset_index(drop=True)
ols_model = sm.OLS(y_train_reset, X_train_const).fit()
print(ols_model.summary())

In [ ]:
# Predictions
y_pred_lr = lr.predict(X_test)
y_pred_lasso = lasso.predict(X_test)
y_pred_rf = rf.predict(X_test)

# Calculate scores
scores = {
    'Model': ['Linear Regression', 'Lasso Regression', 'Random Forest'],
    'R²': [r2_score(y_test, y_pred_lr), r2_score(y_test, y_pred_lasso), r2_score(y_test, y_pred_rf)],
    'RMSE': [
        np.sqrt(mean_squared_error(y_test, y_pred_lr)),
        np.sqrt(mean_squared_error(y_test, y_pred_lasso)),
        np.sqrt(mean_squared_error(y_test, y_pred_rf))
    ]
}

df_scores = pd.DataFrame(scores)
print(df_scores)

## 7. Ridge Regression and Cross-Validation
Adding Ridge regression (L2) and cross-validation to confirm stability.

In [ ]:
# Ridge with built-in CV to find the best alpha
ridge = RidgeCV(alphas=np.logspace(-6, 6, 13), cv=5)
ridge.fit(X_train, y_train)
y_pred_ridge = ridge.predict(X_test)

print(f"Ridge Regression R²: {r2_score(y_test, y_pred_ridge):.4f}")
print(f"Ridge Regression RMSE: {np.sqrt(mean_squared_error(y_test, y_pred_ridge)):.4f}")

# Cross-Validation (K-Fold) on the training set
cv_lr = -np.mean(cross_val_score(lr, X_train, y_train, cv=5, scoring='neg_mean_squared_error'))
cv_ridge = -np.mean(cross_val_score(ridge, X_train, y_train, cv=5, scoring='neg_mean_squared_error'))
cv_lasso = -np.mean(cross_val_score(lasso, X_train, y_train, cv=5, scoring='neg_mean_squared_error'))

print("\n--- Cross-Validation MSE ---")
print(f"Linear: {cv_lr:.2f}")
print(f"Ridge:  {cv_ridge:.2f}")
print(f"Lasso:  {cv_lasso:.2f}")

## 8. Results Visualization
Graphical comparison of errors (RMSE) between models.

In [ ]:
models_list = ['Linear', 'Ridge', 'Lasso', 'Random Forest']
rmse_vals = [
    np.sqrt(mean_squared_error(y_test, y_pred_lr)),
    np.sqrt(mean_squared_error(y_test, y_pred_ridge)),
    np.sqrt(mean_squared_error(y_test, y_pred_lasso)),
    np.sqrt(mean_squared_error(y_test, y_pred_rf))
]

plt.figure(figsize=(10, 6))
plt.bar(models_list, rmse_vals, color=['skyblue', 'lightblue', 'steelblue', 'green'])
plt.title('RMSE Comparison by Model')
plt.ylabel('RMSE (lower is better)')
plt.show()

## 9. Robustness Analysis (Bootstrap)
We use Bootstrap (1000 resamples) to check the stability of the linear model coefficients.

In [ ]:
from sklearn.utils import resample

n_iterations = 1000
n_size = int(len(X_train) * 0.8)
coefs = np.zeros((n_iterations, X_train.shape[1]))

print("Running Bootstrap...")
for i in range(n_iterations):
    X_res, y_res = resample(X_train, y_train, n_samples=n_size, random_state=i)
    model = LinearRegression()
    model.fit(X_res, y_res)
    coefs[i, :] = model.coef_

# Results
bootstrap_res = pd.DataFrame({
    'Feature': X_train.columns,
    'Mean Coeff': np.mean(coefs, axis=0),
    'Std Dev': np.std(coefs, axis=0)
})

print(bootstrap_res)

In [ ]:
# Visualization for a specific feature (e.g., GDP per capita)
# Adapt the name if necessary according to automatic selection
target_feature = "gdp_per_capita"

if target_feature in X.columns:
    idx = list(X.columns).index(target_feature)
    plt.figure(figsize=(8,6))
    plt.hist(coefs[:, idx], bins=30, edgecolor='black', color='orange')
    plt.title(f'Bootstrap Distribution of Coefficient: {target_feature}')
    plt.xlabel('Coefficient Value')
    plt.ylabel('Frequency')
    plt.show()
else:
    print(f"The variable '{target_feature}' was not kept during selection.")